In [5]:
import pathlib
import datetime
import sys
import re
import collections

import IPython.display
print_md = IPython.display.Markdown

import pandas as pd
import numpy as np
import bokeh
import bokeh.plotting as bkh
import bokeh.models as bkhmodels
bkh.output_notebook()

Loading BokehJS ...

# Run1.2i Initial Processing

A look at the logs from coadd and multiband run on Edison in October, 2018

The coadd jobs ran on 16 cores.  Each coadd job ran on one tract and one filter.

Multiband jobs were run on 20 cores.  Each multiband job, ran on 4 patches of one tract, across all filters.

In [6]:
# Location at NERSC
#data_dir=pathlib.Path('/global/cscratch1/sd/heatherk/Run1.2i/processinglogs/edison/coadd/orglogs/')

df = pd.DataFrame()
#path=data_dir+'data.json'
#print(path)
df = df.append(pd.read_json('/global/cscratch1/sd/heatherk/Run1.2i/processinglogs/edison/coadd/orglogs/data.json', lines=True))
df = df.append(pd.read_json('/global/cscratch1/sd/heatherk/Run1.2i/processinglogs/edison/multi/logs/multi.json', lines=True))

    
rows = df.shape[0]
if rows == 0:
    raise Exception("no job records could be loaded")
    
print_md(f'Loaded **{rows}** job accounting records')

Loaded **422** job accounting records

In [7]:
num_rows, num_columns = df.shape
columns = sorted(list(df))
print_md(f'Dataframe size: **{num_rows} rows, {num_columns} columns:**\n\n `{", ".join(columns)}`')

Dataframe size: **422 rows, 13 columns:**

 `band, cores, elapsed_time, elapsed_time_seconds, exit_status, job, max_mem_gbytes, max_mem_kbytes, patch, run, system_time_sec, tract, user_time_sec`

# Looking at min, median, and max elapsed times and memory

## Coadd elapsed time in hours

In [8]:
df.loc[(df['exit_status']==0)&(df['job']=='coadd'),"elapsed_time_seconds"].min()/3600

0.12722222222222221

In [9]:
df.loc[(df['exit_status']==0)&(df['job']=='coadd'),"elapsed_time_seconds"].median()/3600

2.144722222222222

In [10]:
df.loc[(df['exit_status']==0)&(df['job']=='coadd'),"elapsed_time_seconds"].max()/3600

8.8755555555555556

## coadd memory use in gigabytes

In [11]:
df.loc[(df['exit_status']==0)&(df['job']=='coadd'),"max_mem_gbytes"].min()

3.411292

In [12]:
df.loc[(df['exit_status']==0)&(df['job']=='coadd'),"max_mem_gbytes"].median()

4.05598

In [27]:
df.loc[(df['exit_status']==0)&(df['job']=='coadd'),"max_mem_gbytes"].max()

6.1644480000000001

## Multiband elapsed time in hours

In [14]:
df.loc[(df['exit_status']==0)&(df['job']=='multiband'),"elapsed_time_seconds"].median()/3600

0.4865277777777778

In [15]:
df.loc[(df['exit_status']==0)&(df['job']=='multiband'),"elapsed_time_seconds"].max()/3600

2.013611111111111

In [16]:
df.loc[(df['exit_status']==0)&(df['job']=='multiband'),"elapsed_time_seconds"].min()/3600

0.02388888888888889

## Multiband memory use in gigabytes

In [17]:
df.loc[(df['exit_status']==0)&(df['job']=='multiband'),"max_mem_gbytes"].min()

0.52136799999999994

In [18]:
df.loc[(df['exit_status']==0)&(df['job']=='multiband'),"max_mem_gbytes"].median()

1.181486

In [19]:
df.loc[(df['job']=='multiband'),"max_mem_gbytes"].max()

2.2215639999999999

In [20]:
def format_pct_hours(v):
    """Format v to be displayed as a percentile, in hours. Argument v is expected to be seconds.
    """
    return "<1" if v < 3600.0 else f'{int(v/3600.0)}'

def format_pct_min(v):
    """Format v to be displayed as a percentile, in minutes. Argument v is expected to be seconds.
    """
    return "<1" if v < 60.0 else f'{int(v/60.0)}'

def style_figure(fig, x_font_style='bold', x_label_standoff=15, y_font_style='bold', y_label_standoff=15):
    """Style some attributes of a Bokeh figure
    """
    # Figure size (in screen units)
    fig.plot_width = 800
    fig.plot_height = 600
    
    # Autohide figure toolbar
    #fig.toolbar.autohide = True

    # Figure background fill color and alpha
    fig.background_fill_color = 'whitesmoke'
    fig.background_fill_alpha = 0.8
    
    # Axis properties
    fig.xaxis.axis_label_text_font_style = x_font_style
    fig.xaxis.axis_label_standoff =  x_label_standoff
    fig.yaxis.axis_label_text_font_style = y_font_style
    fig.yaxis.axis_label_standoff =  y_label_standoff

In [21]:
def format_failed_pct(v):
    return '{:.0%}'.format(v) if v > 0.0 else 'n/a'

def format_job_count(count):
    return '{:,}'.format(count)

d = {}
bands = df.groupby(['band']).size().sort_values(ascending=False)
for b in bands.index:
    job_count = bands[b]
    succeeded = df.loc[(df['band'] == b) & (df['exit_status'] == 0)].shape[0]
    d[b] = {
        'job_count': job_count,
        'succeeded': succeeded/job_count if job_count > 0 else 0,
        'failed':    (job_count - succeeded)/job_count if job_count > 0 else 0,
    }

table = """
### Percentage of failed jobs (`exit_status != 0`)

| Band | Job Count | Succeeded | Failed |
| ----: | --------: | --------: | -----: |
"""

for k, v in d.items():
    table += f'| **{k}** | {format_job_count(v["job_count"])} | {format_failed_pct(v["succeeded"])} | {format_failed_pct(v["failed"])} |\n'

table += """
*The table above shows the fraction of succeeded and failed jobs per band. A job is considered failed if its `exit_status` is not zero.*
*g^i^r^u^y^z indicates multiband, while the single band jobs were coadds.*
"""    
print_md(table)


### Percentage of failed jobs (`exit_status != 0`)

| Band | Job Count | Succeeded | Failed |
| ----: | --------: | --------: | -----: |
| **g^i^r^u^y^z** | 320 | 100% | n/a |
| **y** | 21 | 100% | n/a |
| **r** | 21 | 100% | n/a |
| **z** | 20 | 100% | n/a |
| **u** | 20 | 95% | 5% |
| **i** | 20 | 100% | n/a |

*The table above shows the fraction of succeeded and failed jobs per band. A job is considered failed if its `exit_status` is not zero.*
*g^i^r^u^y^z indicates multiband, while the single band jobs were coadds.*


In [23]:
fig = bkh.figure(
    title = 'DISTRIBUTION OF JOB EXECUTION WALLCLOCK TIME COADDS',
    x_axis_label = 'execution wallclock time (hours)',
    y_axis_label = 'job count'
)


style_figure(fig)
color, alpha = 'darkseagreen', 0.7
df.loc[(df['exit_status']==0)&(df['job']=='coadd'),"elapsed_time_seconds"]
hist, edges = np.histogram(df.loc[(df['exit_status']==0)&(df['job']=='coadd'),'elapsed_time_seconds']/3600, density=False, bins=40)
fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=alpha, fill_color=color, line_color=color)
fig.yaxis.formatter = bkhmodels.formatters.NumeralTickFormatter(format="0,0")
fig.add_tools(bkhmodels.HoverTool(
    tooltips = [
        ('jobs', '@top{0,000}'),
        ('low',  '@left{0.0} hours'),
        ('high', '@right{0.0} hours'),
    ],
    mode = 'mouse',
))
bkh.show(fig)

In [24]:
fig = bkh.figure(
    title = 'DISTRIBUTION OF JOB EXECUTION WALLCLOCK TIME Multiband',
    x_axis_label = 'execution wallclock time (hours)',
    y_axis_label = 'job count'
)


style_figure(fig)
color, alpha = 'darkseagreen', 0.7
hist, edges = np.histogram(df.loc[(df['exit_status']==0)&(df['job']=='multiband'),'elapsed_time_seconds']/3600, density=False, bins=40)
fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=alpha, fill_color=color, line_color=color)
fig.yaxis.formatter = bkhmodels.formatters.NumeralTickFormatter(format="0,0")
fig.add_tools(bkhmodels.HoverTool(
    tooltips = [
        ('jobs', '@top{0,000}'),
        ('low',  '@left{0.0} hours'),
        ('high', '@right{0.0} hours'),
    ],
    mode = 'mouse',
))
bkh.show(fig)

In [25]:
successful_jobs = df.loc[(df['exit_status'] == 0)]

# Create the figure
fig = bkh.figure(
    title = 'DISTRIBUTION OF MEMORY UTILIZATION',
    x_axis_label = 'gigabyte',
    y_axis_label = 'job count'
)
style_figure(fig)

# Build a histogram per queue
colors = ('dodgerblue', 'darkmagenta', 'mediumseagreen', 'orange', 'crimson', 'sienna')
bands = successful_jobs.groupby(['band']).size().sort_values(ascending=False)
for b, color in zip(bands.index, colors):
    # Select the jobs in this band which wallclock time was higher than the 5th percentile,
    # to exclude jobs which may have terminated early
    job_set = successful_jobs.loc[successful_jobs['band'] == b]
    wallclock_threshold = np.percentile(job_set['elapsed_time_seconds'], [5])[0]
    memory = job_set.loc[job_set['elapsed_time_seconds'] >= wallclock_threshold, 'max_mem_gbytes']
    hist, edges = np.histogram(memory, density=False, bins=20)
    fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha= 0.5, fill_color=color, line_color=color, legend=b)

# Finalize the figure
fig.legend.click_policy = 'hide'
fig.yaxis.formatter = bkhmodels.formatters.NumeralTickFormatter(format="0,0")
fig.add_tools(bkhmodels.HoverTool(
    tooltips = [
        ('jobs', '@top{0,000}'),
        ('low',  '@left{0.0} GB'),
        ('high', '@right{0.0} GB'),
    ],
    mode = 'mouse',
))
bkh.show(fig)
print_md(f"""*The figure above shows the distribution of RAM the jobs (in gigabyte). Click on the legend to hide / show information.*""")

*The figure above shows the distribution of RAM the jobs (in gigabyte). Click on the legend to hide / show information.*

In [26]:
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_csv(sep="|", index=False)))

pandas_df_to_markdown_table(df)

band|cores|elapsed_time|elapsed_time_seconds|exit_status|job|max_mem_gbytes|max_mem_kbytes|patch|run|system_time_sec|tract|user_time_sec
---|---|---|---|---|---|---|---|---|---|---|---|---
y|16|22:27.58|1347|0|coadd|4.006448|4006448|NA|run1.2i|404.19|4433|9307.16
y|16|22:27.58|1347|0|coadd|4.006448|4006448|NA|run1.2i|404.19|4433|9307.16
i|16|1:24:25|5065|0|coadd|4.336412|4336412|NA|run1.2i|1122.12|4429|49278.06
u|16|2:27:31|8851|0|coadd|3.586292|3586292|NA|run1.2i|2533.26|4638|114673.41
u|16|37:31.87|2251|0|coadd|4.206476|4206476|NA|run1.2i|528.76|5063|22150.09
r|16|6:47:52|24472|0|coadd|3.895488|3895488|NA|run1.2i|8382.72|4850|325372.11
r|16|8:57.20|537|0|coadd|4.27948|4279480|NA|run1.2i|256.24|4433|5861.14
i|16|2:53:42|10422|0|coadd|5.3291|5329100|NA|run1.2i|2412.89|4430|111217.66
r|16|3:56:49|14209|0|coadd|3.63608|3636080|NA|run1.2i|1660.6|5064|88147.46
z|16|21:12.62|1272|0|coadd|4.47256|4472560|NA|run1.2i|596.94|5062|16570.41
u|16|1:15:29|4529|0|coadd|3.576432|3576432|NA|run1.2i|1292.53|4637|54591.1
z|16|38:45.11|2325|0|coadd|4.05598|4055980|NA|run1.2i|900.15|4852|27986.86
y|16|2:55:05|10505|0|coadd|4.097744|4097744|NA|run1.2i|1084.7|5063|48370.99
i|16|29:57.52|1797|0|coadd|4.366884|4366884|NA|run1.2i|533.06|4636|16025.51
z|16|1:27:12|5232|0|coadd|4.314948|4314948|NA|run1.2i|1642.72|4848|69696.36
z|16|2:24:32|8672|0|coadd|4.316676|4316676|NA|run1.2i|2967.04|4640|114068.22
y|16|7:11:01|25861|0|coadd|3.672328|3672328|NA|run1.2i|5397.28|4849|209140.06
z|16|4:45:33|17133|0|coadd|3.619552|3619552|NA|run1.2i|2638.34|5065|118295.11
z|16|1:53:52|6832|0|coadd|3.775868|3775868|NA|run1.2i|2075.43|4430|90292.23
y|16|4:15:43|15343|0|coadd|4.289804|4289804|NA|run1.2i|1814.74|5064|82310.02
r|16|6:35:18|23718|0|coadd|3.725324|3725324|NA|run1.2i|4890.06|4851|204997.67
i|16|4:02:12|14532|0|coadd|3.755284|3755284|NA|run1.2i|4356.29|4849|178838.4
z|16|8:10:06|29406|0|coadd|3.68738|3687380|NA|run1.2i|7151.85|4638|279494.12
y|16|3:39:37|13177|0|coadd|6.164448|6164448|NA|run1.2i|2128.06|4640|89561.26
r|16|5:45:03|20703|0|coadd|3.70442|3704420|NA|run1.2i|3825.52|4637|165294.16
y|16|48:08.08|2888|0|coadd|5.538468|5538468|NA|run1.2i|595.79|4636|16485.55
r|16|7:55:15|28515|0|coadd|3.8860200000000003|3886020|NA|run1.2i|10344.26|4638|374335.23
u|16|25:11.54|1511|0|coadd|4.732048|4732048|NA|run1.2i|332.62|5066|9801.47
z|16|1:25:50|5150|0|coadd|5.311536|5311536|NA|run1.2i|1429.79|5063|66640.09
u|16|56:11.96|3371|0|coadd|4.13558|4135580|NA|run1.2i|808.22|5065|39919.63
z|16|41:58.41|2518|0|coadd|4.63696|4636960|NA|run1.2i|995.14|4429|35651.96
u|16|1:02:54|3774|0|coadd|4.843652|4843652|NA|run1.2i|822.3|5064|43905.97
y|16|3:13:17|11597|0|coadd|4.3492239999999995|4349224|NA|run1.2i|1304.76|5065|62002.31
y|18|6:26:49|23209|0|coadd|3.82816|3828160|NA|run1.2i|11506.7|4638|332923.97
u|16|45:11.62|2711|0|coadd|5.161896|5161896|NA|run1.2i|583.53|4640|22965.69
i|16|4:15:23|15323|0|coadd|3.7135920000000002|3713592|NA|run1.2i|5059.09|4851|195599.52
i|16|1:44:07|6247|0|coadd|5.08646|5086460|NA|run1.2i|1637.8|5065|77461.26
z|16|2:57:12|10632|0|coadd|3.560968|3560968|NA|run1.2i|3415.64|4637|139836.37
i|16|32:47.19|1967|0|coadd|4.648012|4648012|NA|run1.2i|871.12|4852|25574.27
z|16|16:43.42|1003|0|coadd|4.392464|4392464|NA|run1.2i|498.46|4636|13467.68
y|16|1:53:27|6807|0|coadd|5.284536|5284536|NA|run1.2i|1204.44|4429|41123.0
r|16|2:42:21|9741|0|coadd|3.875264|3875264|NA|run1.2i|3302.54|4432|117575.58
r|16|2:42:21|9741|0|coadd|3.875264|3875264|NA|run1.2i|3302.54|4432|117575.58
r|16|28:47.21|1727|0|coadd|4.557616|4557616|NA|run1.2i|274.34|5062|7327.59
u|16|48:24.73|2904|0|coadd|4.727144|4727144|NA|run1.2i|787.08|4432|29851.56
y|16|59:27.67|3567|0|coadd|4.373104|4373104|NA|run1.2i|719.37|4852|18786.91
z|16|8:21:18|30078|0|coadd|3.7747640000000002|3774764|NA|run1.2i|8402.11|4639|308373.1
u|16|1:11:01|4261|0|coadd|3.411292|3411292|NA|run1.2i|1089.14|4851|51493.32
y|16|5:56:15|21375|0|coadd|3.652732|3652732|NA|run1.2i|4469.48|4637|160239.06
i|16|3:15:27|11727|0|coadd|3.611092|3611092|NA|run1.2i|3514.48|4637|142247.3
y|16|2:51:37|10297|0|coadd|3.901124|3901124|NA|run1.2i|2387.6|4432|86256.91
r|16|4:10:59|15059|0|coadd|3.635608|3635608|NA|run1.2i|1888.83|5065|97158.9
u|16|16:03.70|963|0|coadd|4.118992|4118992|NA|run1.2i|317.59|4636|7736.06
i|16|1:28:42|5322|0|coadd|4.265908|4265908|NA|run1.2i|720.93|5066|26246.67
z|16|2:27:26|8846|0|coadd|3.604936|3604936|NA|run1.2i|2756.0|4431|115714.65
i|16|47:04.55|2824|0|coadd|4.18728|4187280|NA|run1.2i|770.16|5063|38194.4
r|16|1:02:55|3775|0|coadd|3.950436|3950436|NA|run1.2i|816.06|4852|24974.74
i|16|1:32:44|5564|0|coadd|3.686368|3686368|NA|run1.2i|1474.04|5064|75851.64
u|16|1:35:13|5713|0|coadd|3.525484|3525484|NA|run1.2i|1375.91|4849|70869.87
r|16|2:00:09|7209|0|coadd|4.168948|4168948|NA|run1.2i|652.42|5063|33792.88
y|16|8:39:54|31194|0|coadd|3.787432|3787432|NA|run1.2i|7759.2|4850|284365.06
y|16|8:52:32|31952|0|coadd|3.739132|3739132|NA|run1.2i|8084.98|4639|300726.94
u|16|29:05.90|1745|0|coadd|5.06544|5065440|NA|run1.2i|419.56|4848|15093.35
z|16|7:38.91|458|0|coadd|4.384656|4384656|NA|run1.2i|232.02|4433|6179.14
z|16|7:03:32|25412|0|coadd|3.755624|3755624|NA|run1.2i|5403.24|4849|224316.16
i|16|6:54:46|24886|0|coadd|3.7791959999999998|3779196|NA|run1.2i|8341.61|4638|303882.87
r|16|7:06:53|25613|0|coadd|5.270148|5270148|NA|run1.2i|5666.48|4431|222900.63
y|16|4:58:46|17926|0|coadd|5.33402|5334020|NA|run1.2i|3314.85|4430|132489.65
z|16|51:38.01|3098|0|coadd|5.330588|5330588|NA|run1.2i|910.39|5066|39895.66
u|16|16:22.30|982|0|coadd|5.546056|5546056|NA|run1.2i|293.35|4852|6784.36
r|16|50:57.24|3057|0|coadd|4.3956479999999996|4395648|NA|run1.2i|617.22|4636|18263.06
z|16|1:31:15|5475|0|coadd|4.061412|4061412|NA|run1.2i|1840.91|4432|71736.18
i|16|11:03.15|663|0|coadd|3.9417400000000002|3941740|NA|run1.2i|297.43|4433|6613.44
u|16|2:08:41|7721|0|coadd|3.5789|3578900|NA|run1.2i|2041.0|4850|97945.64
u|16|1:11:29|4289|0|coadd|4.004608|4004608|NA|run1.2i|1126.3|4430|52126.15
i|16|2:30:11|9011|0|coadd|3.86992|3869920|NA|run1.2i|3234.71|4640|123192.78
i|16|9:26.18|566|0|coadd|3.7152760000000002|3715276|NA|run1.2i|232.18|5062|5805.82
u|16|0:09.45|9|2|coadd|0.350784|350784|NA|run1.2i|3.12|5062|2.96
y|16|5:39:03|20343|0|coadd|5.275564|5275564|NA|run1.2i|3999.22|4431|159867.11
i|16|1:03:22|3802|0|coadd|5.302244|5302244|NA|run1.2i|1179.76|4848|51852.02
i|16|2:15:14|8114|0|coadd|4.03698|4036980|NA|run1.2i|2323.52|4432|81115.91
i|16|3:10:32|11432|0|coadd|3.45142|3451420|NA|run1.2i|3686.86|4431|137626.54
i|16|6:32:34|23554|0|coadd|3.750728|3750728|NA|run1.2i|7903.27|4850|293180.95
u|16|2:02:28|7348|0|coadd|3.50114|3501140|NA|run1.2i|2182.48|4639|94537.54
i|16|6:48:16|24496|0|coadd|3.742784|3742784|NA|run1.2i|8724.08|4639|302432.28
r|16|40:02.00|2402|0|coadd|3.977456|3977456|NA|run1.2i|724.83|5066|33945.82
r|16|2:33:06|9186|0|coadd|4.382144|4382144|NA|run1.2i|1458.06|4429|57723.36
r|16|6:09:57|22197|0|coadd|5.291624|5291624|NA|run1.2i|4692.88|4430|189765.2
z|16|4:39:14|16754|0|coadd|3.578476|3578476|NA|run1.2i|2736.48|5064|127415.35
z|16|4:39:14|16754|0|coadd|3.578476|3578476|NA|run1.2i|2736.48|5064|127415.35
u|16|35:17.84|2117|0|coadd|5.537704|5537704|NA|run1.2i|524.08|4429|20454.51
y|16|5:20:20|19220|0|coadd|3.675816|3675816|NA|run1.2i|3171.06|4851|138412.06
y|16|3:11:26|11486|0|coadd|4.701916|4701916|NA|run1.2i|1537.25|4848|57987.25
r|16|6:17:25|22645|0|coadd|3.747692|3747692|NA|run1.2i|4239.74|4849|185724.05
y|16|21:58.89|1318|0|coadd|4.358992|4358992|NA|run1.2i|419.93|5062|11372.06
z|16|7:05:13|25513|0|coadd|3.721864|3721864|NA|run1.2i|5849.85|4851|229084.92
u|16|9:17.39|557|0|coadd|4.767528|4767528|NA|run1.2i|225.5|4433|4971.09
r|16|4:51:44|17504|0|coadd|5.229924|5229924|NA|run1.2i|3452.77|4640|129790.26
r|16|8:19:49|29989|0|coadd|3.850056|3850056|NA|run1.2i|11155.82|4639|398443.83
u|16|1:35:07|5707|0|coadd|4.082404|4082404|NA|run1.2i|1797.4|4431|72200.66
y|16|55:21.84|3321|0|coadd|5.5208200000000005|5520820|NA|run1.2i|631.17|5066|19325.39
r|16|2:27:56|8876|0|coadd|5.423412|5423412|NA|run1.2i|1052.84|4848|46281.51
g^i^r^u^y^z|20|42:04.57|2524|0|multiband|1.398536|1398536|2,4^2,5^2,6^2,7|run1.2i|1032.61|5064|30510.44
g^i^r^u^y^z|20|2:26.50|146|0|multiband|0.521528|521528|0,0^0,1^0,2^0,3|run1.2i|106.45|4640|1362.1
g^i^r^u^y^z|20|26:32.48|1592|0|multiband|1.159672|1159672|0,4^0,5^0,6^0,7|run1.2i|698.7|4638|25405.89
g^i^r^u^y^z|20|27:15.14|1635|0|multiband|1.163676|1163676|6,4^6,5^6,6^6,7|run1.2i|783.37|4638|26594.94
g^i^r^u^y^z|20|31:40.83|1900|0|multiband|1.60576|1605760|7,0^7,1^7,2^7,3|run1.2i|1166.72|4430|34225.62
g^i^r^u^y^z|20|56:32.93|3392|0|multiband|1.8509479999999998|1850948|2,4^2,5^2,6^2,7|run1.2i|1529.14|5063|46617.86
g^i^r^u^y^z|20|1:02:34|3754|0|multiband|1.423448|1423448|6,4^6,5^6,6^6,7|run1.2i|1991.58|4431|68153.34
g^i^r^u^y^z|20|22:39.62|1359|0|multiband|1.14982|1149820|7,0^7,1^7,2^7,3|run1.2i|705.52|4638|21335.46
g^i^r^u^y^z|20|51:27.32|3087|0|multiband|1.4157440000000001|1415744|1,0^1,1^1,2^1,3|run1.2i|1560.24|4637|55037.3
g^i^r^u^y^z|20|24:50.77|1490|0|multiband|1.2086000000000001|1208600|3,4^3,5^3,6^3,7|run1.2i|799.24|4850|24862.88
g^i^r^u^y^z|20|18:35.14|1115|0|multiband|1.146408|1146408|6,4^6,5^6,6^6,7|run1.2i|672.45|4637|19755.2
g^i^r^u^y^z|20|39:12.42|2352|0|multiband|1.3792879999999998|1379288|7,4^7,5^7,6^7,7|run1.2i|934.56|5064|26354.01
g^i^r^u^y^z|20|1:00:18|3618|0|multiband|2.087736|2087736|6,0^6,1^6,2^6,3|run1.2i|1266.14|5063|50423.64
g^i^r^u^y^z|20|1:34:15|5655|0|multiband|2.011|2011000|6,4^6,5^6,6^6,7|run1.2i|2289.04|4432|104480.9
g^i^r^u^y^z|20|8:38.61|518|0|multiband|1.1835200000000001|1183520|0,0^0,1^0,2^0,3|run1.2i|270.55|4432|7519.76
g^i^r^u^y^z|20|55:45.33|3345|0|multiband|2.141028|2141028|5,0^5,1^5,2^5,3|run1.2i|1815.04|4432|60264.94
g^i^r^u^y^z|20|1:45.48|105|0|multiband|0.521636|521636|6,0^6,1^6,2^6,3|run1.2i|91.66|4433|1062.21
g^i^r^u^y^z|20|32:56.37|1976|0|multiband|1.6006200000000002|1600620|6,4^6,5^6,6^6,7|run1.2i|477.87|4852|12530.5
g^i^r^u^y^z|20|1:33.83|93|0|multiband|0.521624|521624|4,0^4,1^4,2^4,3|run1.2i|88.26|4433|1047.1
g^i^r^u^y^z|20|24:00.34|1440|0|multiband|1.15696|1156960|2,4^2,5^2,6^2,7|run1.2i|706.61|4637|22154.5
g^i^r^u^y^z|20|32:53.65|1973|0|multiband|1.179452|1179452|6,0^6,1^6,2^6,3|run1.2i|1048.08|4850|34145.16
g^i^r^u^y^z|20|39:08.31|2348|0|multiband|1.3789639999999999|1378964|3,4^3,5^3,6^3,7|run1.2i|922.27|5064|25006.87
g^i^r^u^y^z|20|54:57.41|3297|0|multiband|2.102252|2102252|0,0^0,1^0,2^0,3|run1.2i|1915.57|4429|59649.88
g^i^r^u^y^z|20|34:15.38|2055|0|multiband|2.0922|2092200|2,0^2,1^2,2^2,3|run1.2i|1082.86|4429|36970.06
g^i^r^u^y^z|20|38:18.80|2298|0|multiband|1.1579679999999999|1157968|6,0^6,1^6,2^6,3|run1.2i|791.22|4851|24523.92
g^i^r^u^y^z|20|6:16.55|376|0|multiband|0.521408|521408|6,0^6,1^6,2^6,3|run1.2i|123.12|5062|1824.4
g^i^r^u^y^z|20|17:28.14|1048|0|multiband|1.166356|1166356|4,4^4,5^4,6^4,7|run1.2i|573.96|4637|18506.02
g^i^r^u^y^z|20|58:51.80|3531|0|multiband|1.64256|1642560|2,0^2,1^2,2^2,3|run1.2i|1456.72|5065|49398.04
g^i^r^u^y^z|20|6:16.28|376|0|multiband|0.521612|521612|4,0^4,1^4,2^4,3|run1.2i|122.7|4852|1752.18
g^i^r^u^y^z|20|1:20:52|4852|0|multiband|1.6807400000000001|1680740|0,0^0,1^0,2^0,3|run1.2i|2154.38|5064|73979.77
g^i^r^u^y^z|20|33:51.92|2031|0|multiband|1.425456|1425456|6,0^6,1^6,2^6,3|run1.2i|516.93|4852|14240.93
g^i^r^u^y^z|20|11:56.50|716|0|multiband|1.6002200000000002|1600220|7,4^7,5^7,6^7,7|run1.2i|380.47|4433|11948.11
g^i^r^u^y^z|20|21:42.47|1302|0|multiband|1.1521919999999999|1152192|2,0^2,1^2,2^2,3|run1.2i|706.9|4639|22198.12
g^i^r^u^y^z|20|6:10.83|370|0|multiband|0.521568|521568|3,0^3,1^3,2^3,3|run1.2i|121.68|5062|1731.64
g^i^r^u^y^z|20|1:02:58|3778|0|multiband|2.058732|2058732|7,0^7,1^7,2^7,3|run1.2i|1422.3|4852|53147.07
g^i^r^u^y^z|20|1:58.44|118|0|multiband|0.5215200000000001|521520|6,0^6,1^6,2^6,3|run1.2i|100.57|4848|1327.26
g^i^r^u^y^z|20|36:09.75|2169|0|multiband|1.634812|1634812|3,0^3,1^3,2^3,3|run1.2i|1087.9|4431|38939.03
g^i^r^u^y^z|20|37:02.38|2222|0|multiband|1.136096|1136096|7,4^7,5^7,6^7,7|run1.2i|728.88|4851|22465.83
g^i^r^u^y^z|20|53:26.76|3206|0|multiband|1.798068|1798068|0,0^0,1^0,2^0,3|run1.2i|1219.01|5062|43407.79
g^i^r^u^y^z|20|30:27.86|1827|0|multiband|1.125068|1125068|3,4^3,5^3,6^3,7|run1.2i|550.16|5065|14620.7
g^i^r^u^y^z|20|26:19.65|1579|0|multiband|0.9971000000000001|997100|0,4^0,5^0,6^0,7|run1.2i|381.82|4851|9519.06
g^i^r^u^y^z|20|20:39.86|1239|0|multiband|1.146968|1146968|7,4^7,5^7,6^7,7|run1.2i|616.9|4850|20654.72
g^i^r^u^y^z|20|37:31.22|2251|0|multiband|1.44116|1441160|1,0^1,1^1,2^1,3|run1.2i|1227.55|4430|40606.73
g^i^r^u^y^z|20|1:06:59|4019|0|multiband|2.16866|2168660|7,0^7,1^7,2^7,3|run1.2i|2042.99|4432|71698.21
g^i^r^u^y^z|20|24:53.94|1493|0|multiband|1.158568|1158568|0,4^0,5^0,6^0,7|run1.2i|758.48|4849|25323.77
g^i^r^u^y^z|20|25:12.58|1512|0|multiband|1.171424|1171424|5,4^5,5^5,6^5,7|run1.2i|755.05|4639|25228.9
g^i^r^u^y^z|20|20:27.93|1227|0|multiband|1.170536|1170536|4,0^4,1^4,2^4,3|run1.2i|706.93|4849|21996.13
g^i^r^u^y^z|20|36:08.75|2168|0|multiband|1.341512|1341512|0,4^0,5^0,6^0,7|run1.2i|794.9|5065|21867.59
g^i^r^u^y^z|20|36:05.19|2165|0|multiband|1.630372|1630372|2,0^2,1^2,2^2,3|run1.2i|1328.05|4431|39069.35
g^i^r^u^y^z|20|44:53.09|2693|0|multiband|1.397776|1397776|1,4^1,5^1,6^1,7|run1.2i|1176.54|5064|33944.17
g^i^r^u^y^z|20|21:20.78|1280|0|multiband|1.160592|1160592|5,0^5,1^5,2^5,3|run1.2i|680.0|4849|21824.73
g^i^r^u^y^z|20|22:25.66|1345|0|multiband|1.3607559999999999|1360756|6,4^6,5^6,6^6,7|run1.2i|728.05|4849|22811.86
g^i^r^u^y^z|20|35:57.80|2157|0|multiband|1.615864|1615864|3,4^3,5^3,6^3,7|run1.2i|1002.08|4640|37629.16
g^i^r^u^y^z|20|24:12.71|1452|0|multiband|1.6823519999999998|1682352|0,4^0,5^0,6^0,7|run1.2i|767.25|4432|24502.66
g^i^r^u^y^z|20|59:42.74|3582|0|multiband|1.404576|1404576|0,0^0,1^0,2^0,3|run1.2i|1923.21|4637|63596.14
g^i^r^u^y^z|20|7:21.53|441|0|multiband|1.0323|1032300|0,0^0,1^0,2^0,3|run1.2i|207.84|4851|5991.09
g^i^r^u^y^z|20|42:31.81|2551|0|multiband|1.3953039999999999|1395304|5,0^5,1^5,2^5,3|run1.2i|1388.83|4637|46190.4
g^i^r^u^y^z|20|1:13:44|4424|0|multiband|1.678036|1678036|2,0^2,1^2,2^2,3|run1.2i|2035.6|5064|67461.02
g^i^r^u^y^z|20|51:48.08|3108|0|multiband|1.817172|1817172|3,4^3,5^3,6^3,7|run1.2i|1429.42|5063|41092.56
g^i^r^u^y^z|20|37:40.26|2260|0|multiband|1.146744|1146744|5,4^5,5^5,6^5,7|run1.2i|830.1|4851|23400.38
g^i^r^u^y^z|20|1:48.85|108|0|multiband|0.521528|521528|7,4^7,5^7,6^7,7|run1.2i|95.85|4636|1199.99
g^i^r^u^y^z|20|6:01.44|361|0|multiband|0.521532|521532|4,4^4,5^4,6^4,7|run1.2i|121.03|4852|1742.12
g^i^r^u^y^z|20|6:12.69|372|0|multiband|0.521532|521532|5,0^5,1^5,2^5,3|run1.2i|125.26|4852|1784.69
g^i^r^u^y^z|20|38:03.43|2283|0|multiband|1.8116919999999999|1811692|6,4^6,5^6,6^6,7|run1.2i|849.86|5063|24232.65
g^i^r^u^y^z|20|21:59.98|1319|0|multiband|1.15994|1159940|7,0^7,1^7,2^7,3|run1.2i|680.9|4639|21563.86
g^i^r^u^y^z|20|47:29.33|2849|0|multiband|1.86814|1868140|2,4^2,5^2,6^2,7|run1.2i|1368.43|4848|51772.19
g^i^r^u^y^z|20|11:42.36|702|0|multiband|1.82726|1827260|5,4^5,5^5,6^5,7|run1.2i|398.64|4429|11960.7
g^i^r^u^y^z|20|1:07:01|4021|0|multiband|1.7301959999999998|1730196|1,4^1,5^1,6^1,7|run1.2i|1884.15|4430|72417.21
g^i^r^u^y^z|20|6:15.35|375|0|multiband|0.521524|521524|0,0^0,1^0,2^0,3|run1.2i|118.78|5066|1781.39
g^i^r^u^y^z|20|29:45.74|1785|0|multiband|1.120216|1120216|4,4^4,5^4,6^4,7|run1.2i|548.1|5065|13915.08
g^i^r^u^y^z|20|31:45.59|1905|0|multiband|1.126152|1126152|6,4^6,5^6,6^6,7|run1.2i|643.59|5065|16428.96
g^i^r^u^y^z|20|46:13.19|2773|0|multiband|1.398348|1398348|7,4^7,5^7,6^7,7|run1.2i|997.29|4431|49384.62
g^i^r^u^y^z|20|32:37.89|1957|0|multiband|1.38514|1385140|6,0^6,1^6,2^6,3|run1.2i|1097.88|4637|35271.73
g^i^r^u^y^z|20|4:02.32|242|0|multiband|0.5214639999999999|521464|7,4^7,5^7,6^7,7|run1.2i|115.85|5062|1592.31
g^i^r^u^y^z|20|57:05.06|3425|0|multiband|1.58834|1588340|0,0^0,1^0,2^0,3|run1.2i|1126.94|5065|47182.18
g^i^r^u^y^z|20|23:47.82|1427|0|multiband|1.221312|1221312|6,4^6,5^6,6^6,7|run1.2i|735.46|4850|23745.35
g^i^r^u^y^z|20|19:56.19|1196|0|multiband|1.150476|1150476|1,0^1,1^1,2^1,3|run1.2i|630.78|4639|20434.74
g^i^r^u^y^z|20|1:42.76|102|0|multiband|0.52164|521640|2,4^2,5^2,6^2,7|run1.2i|91.58|4433|1053.95
g^i^r^u^y^z|20|1:19:26|4766|0|multiband|1.9978159999999998|1997816|7,4^7,5^7,6^7,7|run1.2i|1934.34|4432|89043.96
g^i^r^u^y^z|20|32:21.58|1941|0|multiband|1.458848|1458848|3,0^3,1^3,2^3,3|run1.2i|519.68|5066|13747.34
g^i^r^u^y^z|20|50:40.51|3040|0|multiband|1.410228|1410228|5,4^5,5^5,6^5,7|run1.2i|1272.9|4431|53282.35
g^i^r^u^y^z|20|17:41.63|1061|0|multiband|1.149984|1149984|5,0^5,1^5,2^5,3|run1.2i|551.64|4639|17376.91
g^i^r^u^y^z|20|21:27.66|1287|0|multiband|1.156052|1156052|5,4^5,5^5,6^5,7|run1.2i|755.48|4637|22940.66
g^i^r^u^y^z|20|23:36.04|1416|0|multiband|1.162064|1162064|6,4^6,5^6,6^6,7|run1.2i|718.97|4639|23186.23
g^i^r^u^y^z|20|1:05:35|3935|0|multiband|1.657136|1657136|5,0^5,1^5,2^5,3|run1.2i|1678.2|5065|56876.45
g^i^r^u^y^z|20|1:46.75|106|0|multiband|0.52166|521660|5,4^5,5^5,6^5,7|run1.2i|94.06|4636|1125.54
g^i^r^u^y^z|20|57:03.04|3423|0|multiband|1.707104|1707104|3,4^3,5^3,6^3,7|run1.2i|1620.29|4430|62357.3
g^i^r^u^y^z|20|26:38.08|1598|0|multiband|1.179088|1179088|6,0^6,1^6,2^6,3|run1.2i|868.94|4849|27665.53
g^i^r^u^y^z|20|16:35.06|995|0|multiband|1.065252|1065252|7,4^7,5^7,6^7,7|run1.2i|636.1|4637|17457.41
g^i^r^u^y^z|20|36:54.75|2214|0|multiband|1.651484|1651484|1,0^1,1^1,2^1,3|run1.2i|1139.24|4431|40038.07
g^i^r^u^y^z|20|26:23.52|1583|0|multiband|1.168336|1168336|3,0^3,1^3,2^3,3|run1.2i|866.61|4849|27112.84
g^i^r^u^y^z|20|30:27.87|1827|0|multiband|1.1132|1113200|7,4^7,5^7,6^7,7|run1.2i|554.04|5065|14895.99
g^i^r^u^y^z|20|1:16:33|4593|0|multiband|2.129984|2129984|4,0^4,1^4,2^4,3|run1.2i|1700.95|5063|70620.8
g^i^r^u^y^z|20|44:58.51|2698|0|multiband|1.803016|1803016|7,0^7,1^7,2^7,3|run1.2i|854.47|5063|32848.32
g^i^r^u^y^z|20|55:55.57|3355|0|multiband|1.41774|1417740|2,4^2,5^2,6^2,7|run1.2i|1811.15|4431|58510.4
g^i^r^u^y^z|20|29:22.77|1762|0|multiband|1.176804|1176804|5,4^5,5^5,6^5,7|run1.2i|909.15|4638|28938.77
g^i^r^u^y^z|20|24:50.59|1490|0|multiband|1.164208|1164208|3,0^3,1^3,2^3,3|run1.2i|767.42|4850|24965.71
g^i^r^u^y^z|20|4:43.88|283|0|multiband|0.521404|521404|3,4^3,5^3,6^3,7|run1.2i|118.43|5062|1616.41
g^i^r^u^y^z|20|1:04:28|3868|0|multiband|1.64396|1643960|3,0^3,1^3,2^3,3|run1.2i|1678.56|5065|55351.55
g^i^r^u^y^z|20|1:34.94|94|0|multiband|0.521612|521612|6,4^6,5^6,6^6,7|run1.2i|91.36|4429|1083.8
g^i^r^u^y^z|20|51:26.11|3086|0|multiband|1.957116|1957116|0,4^0,5^0,6^0,7|run1.2i|1484.42|4636|54037.3
g^i^r^u^y^z|20|31:56.33|1916|0|multiband|1.3267959999999999|1326796|0,4^0,5^0,6^0,7|run1.2i|596.28|5062|15382.32
g^i^r^u^y^z|20|33:02.76|1982|0|multiband|1.5987239999999998|1598724|4,4^4,5^4,6^4,7|run1.2i|1221.51|4848|35289.83
g^i^r^u^y^z|20|23:37.54|1417|0|multiband|1.1635|1163500|4,0^4,1^4,2^4,3|run1.2i|655.61|4850|23591.32
g^i^r^u^y^z|20|39:51.50|2391|0|multiband|1.831064|1831064|1,4^1,5^1,6^1,7|run1.2i|1162.61|4432|42313.28
g^i^r^u^y^z|20|6:11.05|371|0|multiband|0.521396|521396|0,4^0,5^0,6^0,7|run1.2i|122.31|4852|1792.61
g^i^r^u^y^z|20|49:20.05|2960|0|multiband|1.8104040000000001|1810404|4,0^4,1^4,2^4,3|run1.2i|1116.68|5066|38249.04
g^i^r^u^y^z|20|1:18:39|4719|0|multiband|1.687436|1687436|3,0^3,1^3,2^3,3|run1.2i|2047.35|5064|71831.38
g^i^r^u^y^z|20|6:16.15|376|0|multiband|0.521412|521412|5,4^5,5^5,6^5,7|run1.2i|121.57|5062|1773.28
g^i^r^u^y^z|20|34:53.87|2093|0|multiband|1.134424|1134424|4,4^4,5^4,6^4,7|run1.2i|721.11|4851|19717.31
g^i^r^u^y^z|20|29:59.92|1799|0|multiband|1.122504|1122504|2,4^2,5^2,6^2,7|run1.2i|574.3|5065|14158.24
g^i^r^u^y^z|20|58:38.32|3518|0|multiband|1.8324880000000001|1832488|7,4^7,5^7,6^7,7|run1.2i|1373.44|4852|49515.12
g^i^r^u^y^z|20|36:36.84|2196|0|multiband|1.356352|1356352|0,4^0,5^0,6^0,7|run1.2i|804.13|5064|22296.52
g^i^r^u^y^z|20|1:08:46|4126|0|multiband|1.71706|1717060|0,4^0,5^0,6^0,7|run1.2i|1830.43|4430|74844.96
g^i^r^u^y^z|20|29:06.44|1746|0|multiband|1.365132|1365132|7,0^7,1^7,2^7,3|run1.2i|705.16|4637|31719.95
g^i^r^u^y^z|20|1:37.31|97|0|multiband|0.5215839999999999|521584|4,4^4,5^4,6^4,7|run1.2i|93.07|4433|1151.65
g^i^r^u^y^z|20|1:17:04|4624|0|multiband|1.667828|1667828|6,0^6,1^6,2^6,3|run1.2i|2253.66|5065|69788.07
g^i^r^u^y^z|20|7:30.93|450|0|multiband|0.99482|994820|2,0^2,1^2,2^2,3|run1.2i|215.88|4640|5896.87
g^i^r^u^y^z|20|48:26.70|2906|0|multiband|2.073412|2073412|0,0^0,1^0,2^0,3|run1.2i|1054.34|4636|51465.02
g^i^r^u^y^z|20|1:26.80|86|0|multiband|0.521524|521524|6,4^6,5^6,6^6,7|run1.2i|87.96|4433|1045.61
g^i^r^u^y^z|20|16:41.30|1001|0|multiband|1.132968|1132968|0,4^0,5^0,6^0,7|run1.2i|496.2|4639|15193.32
g^i^r^u^y^z|20|1:17:32|4652|0|multiband|2.221564|2221564|1,0^1,1^1,2^1,3|run1.2i|1705.57|4848|85622.9
g^i^r^u^y^z|20|1:19:43|4783|0|multiband|1.72856|1728560|6,4^6,5^6,6^6,7|run1.2i|2261.97|4640|87325.06
g^i^r^u^y^z|20|1:39.30|99|0|multiband|0.52164|521640|6,0^6,1^6,2^6,3|run1.2i|93.87|4429|1145.98
g^i^r^u^y^z|20|1:51.33|111|0|multiband|0.521524|521524|7,0^7,1^7,2^7,3|run1.2i|92.26|4848|1211.71
g^i^r^u^y^z|20|6:02.48|362|0|multiband|1.174012|1174012|7,0^7,1^7,2^7,3|run1.2i|208.44|4433|5660.71
g^i^r^u^y^z|20|31:53.58|1913|0|multiband|1.126884|1126884|3,4^3,5^3,6^3,7|run1.2i|565.01|4851|15955.01
g^i^r^u^y^z|20|18:50.65|1130|0|multiband|1.700988|1700988|4,0^4,1^4,2^4,3|run1.2i|701.18|4429|19764.69
g^i^r^u^y^z|20|6:15.63|375|0|multiband|0.5214|521400|6,4^6,5^6,6^6,7|run1.2i|121.99|5062|1735.45
g^i^r^u^y^z|20|52:42.54|3162|0|multiband|1.6887919999999998|1688792|4,4^4,5^4,6^4,7|run1.2i|1473.68|4640|56462.29
g^i^r^u^y^z|20|35:15.60|2115|0|multiband|1.420904|1420904|5,0^5,1^5,2^5,3|run1.2i|1262.66|4431|37204.11
g^i^r^u^y^z|20|22:12.38|1332|0|multiband|1.164824|1164824|2,4^2,5^2,6^2,7|run1.2i|726.59|4849|22583.95
g^i^r^u^y^z|20|46:55.34|2815|0|multiband|2.103148|2103148|4,0^4,1^4,2^4,3|run1.2i|1424.5|4432|48030.66
g^i^r^u^y^z|20|1:46.01|106|0|multiband|0.521552|521552|1,0^1,1^1,2^1,3|run1.2i|94.06|4433|1134.52
g^i^r^u^y^z|20|1:46.48|106|0|multiband|0.5215839999999999|521584|5,0^5,1^5,2^5,3|run1.2i|94.83|4636|1151.35
g^i^r^u^y^z|20|37:04.04|2224|0|multiband|2.052736|2052736|4,0^4,1^4,2^4,3|run1.2i|985.2|4848|39622.31
g^i^r^u^y^z|20|52:21.47|3141|0|multiband|1.687164|1687164|4,0^4,1^4,2^4,3|run1.2i|1453.68|4640|56137.48
g^i^r^u^y^z|20|8:18.76|498|0|multiband|1.839944|1839944|1,0^1,1^1,2^1,3|run1.2i|266.82|4636|8011.51
g^i^r^u^y^z|20|29:02.72|1742|0|multiband|1.177108|1177108|2,0^2,1^2,2^2,3|run1.2i|920.68|4849|30293.75
g^i^r^u^y^z|20|25:29.33|1529|0|multiband|1.163756|1163756|1,4^1,5^1,6^1,7|run1.2i|854.66|4850|25689.1
g^i^r^u^y^z|20|1:19:41|4781|0|multiband|2.098488|2098488|6,0^6,1^6,2^6,3|run1.2i|1801.61|5066|72415.05
g^i^r^u^y^z|20|5:16.08|316|0|multiband|0.52156|521560|4,0^4,1^4,2^4,3|run1.2i|115.9|5062|1657.2
g^i^r^u^y^z|20|57:15.80|3435|0|multiband|2.178872|2178872|6,0^6,1^6,2^6,3|run1.2i|1856.18|4432|61922.86
g^i^r^u^y^z|20|1:12:58|4378|0|multiband|1.86666|1866660|0,4^0,5^0,6^0,7|run1.2i|2197.69|5063|64802.59
g^i^r^u^y^z|20|25:20.55|1520|0|multiband|1.260536|1260536|3,4^3,5^3,6^3,7|run1.2i|819.94|4638|26625.98
g^i^r^u^y^z|20|1:45.49|105|0|multiband|0.521604|521604|7,0^7,1^7,2^7,3|run1.2i|99.37|4636|1309.64
g^i^r^u^y^z|20|20:19.87|1219|0|multiband|1.146156|1146156|0,4^0,5^0,6^0,7|run1.2i|584.64|4637|19984.85
g^i^r^u^y^z|20|57:49.87|3469|0|multiband|1.869948|1869948|1,4^1,5^1,6^1,7|run1.2i|1485.63|5063|48624.0
g^i^r^u^y^z|20|52:08.63|3128|0|multiband|2.052592|2052592|7,4^7,5^7,6^7,7|run1.2i|1365.48|5066|41539.94
g^i^r^u^y^z|20|21:47.21|1307|0|multiband|1.369364|1369364|3,0^3,1^3,2^3,3|run1.2i|669.47|4640|23287.11
g^i^r^u^y^z|20|1:55.60|115|0|multiband|0.521564|521564|1,4^1,5^1,6^1,7|run1.2i|94.51|4640|1160.86
g^i^r^u^y^z|20|1:39.01|99|0|multiband|0.521568|521568|3,4^3,5^3,6^3,7|run1.2i|93.21|4636|1107.02
g^i^r^u^y^z|20|1:02:04|3724|0|multiband|1.701528|1701528|7,0^7,1^7,2^7,3|run1.2i|1712.13|4640|68116.94
g^i^r^u^y^z|20|2:00:49|7249|0|multiband|2.167124|2167124|0,0^0,1^0,2^0,3|run1.2i|2710.08|5063|119519.11
g^i^r^u^y^z|20|48:49.85|2929|0|multiband|1.670156|1670156|6,4^6,5^6,6^6,7|run1.2i|1338.09|4430|53162.32
g^i^r^u^y^z|20|28:14.77|1694|0|multiband|1.15586|1155860|0,0^0,1^0,2^0,3|run1.2i|848.13|4638|27244.63
g^i^r^u^y^z|20|37:28.11|2248|0|multiband|1.838184|1838184|3,0^3,1^3,2^3,3|run1.2i|1251.48|4429|40395.23
g^i^r^u^y^z|20|1:26.12|86|0|multiband|0.521496|521496|5,4^5,5^5,6^5,7|run1.2i|88.46|4433|876.22
g^i^r^u^y^z|20|22:41.95|1361|0|multiband|1.16288|1162880|5,0^5,1^5,2^5,3|run1.2i|731.02|4850|23030.57
g^i^r^u^y^z|20|1:45.40|105|0|multiband|0.52156|521560|7,4^7,5^7,6^7,7|run1.2i|96.81|4848|1150.15
g^i^r^u^y^z|20|27:54.27|1674|0|multiband|1.177176|1177176|3,0^3,1^3,2^3,3|run1.2i|877.46|4638|27345.58
g^i^r^u^y^z|20|35:30.55|2130|0|multiband|1.6280999999999999|1628100|0,0^0,1^0,2^0,3|run1.2i|1251.75|4430|38358.16
g^i^r^u^y^z|20|37:40.29|2260|0|multiband|1.15434|1154340|4,0^4,1^4,2^4,3|run1.2i|844.42|4851|24216.02
g^i^r^u^y^z|20|23:27.38|1407|0|multiband|1.15888|1158880|5,0^5,1^5,2^5,3|run1.2i|705.71|4638|22017.4
g^i^r^u^y^z|20|1:42.89|102|0|multiband|0.52166|521660|4,4^4,5^4,6^4,7|run1.2i|91.1|4636|1115.1
g^i^r^u^y^z|20|34:06.14|2046|0|multiband|1.220608|1220608|4,4^4,5^4,6^4,7|run1.2i|681.14|5066|18521.79
g^i^r^u^y^z|20|1:07:10|4030|0|multiband|2.15086|2150860|2,4^2,5^2,6^2,7|run1.2i|1527.37|4429|73913.02
g^i^r^u^y^z|20|54:11.76|3251|0|multiband|2.067732|2067732|5,0^5,1^5,2^5,3|run1.2i|1088.3|5066|43322.05
g^i^r^u^y^z|20|1:17:19|4639|0|multiband|2.18406|2184060|5,4^5,5^5,6^5,7|run1.2i|1750.3|4432|84685.78
g^i^r^u^y^z|20|21:47.73|1307|0|multiband|1.135488|1135488|7,4^7,5^7,6^7,7|run1.2i|739.71|4849|21529.16
g^i^r^u^y^z|20|1:08:15|4095|0|multiband|2.129112|2129112|3,4^3,5^3,6^3,7|run1.2i|1812.21|4432|72595.9
g^i^r^u^y^z|20|33:11.99|1991|0|multiband|1.685048|1685048|1,0^1,1^1,2^1,3|run1.2i|436.83|5062|13569.69
g^i^r^u^y^z|20|25:37.10|1537|0|multiband|1.142276|1142276|0,0^0,1^0,2^0,3|run1.2i|824.03|4850|25861.44
g^i^r^u^y^z|20|6:15.48|375|0|multiband|0.521528|521528|3,4^3,5^3,6^3,7|run1.2i|125.26|4852|1823.19
g^i^r^u^y^z|20|37:44.47|2264|0|multiband|1.633064|1633064|6,0^6,1^6,2^6,3|run1.2i|1384.03|4430|40737.25
g^i^r^u^y^z|20|1:16:27|4587|0|multiband|2.145436|2145436|4,4^4,5^4,6^4,7|run1.2i|1781.58|4432|82298.64
g^i^r^u^y^z|20|1:42.40|102|0|multiband|0.5216959999999999|521696|2,0^2,1^2,2^2,3|run1.2i|93.89|4636|1117.9
g^i^r^u^y^z|20|27:48.60|1668|0|multiband|1.466084|1466084|1,4^1,5^1,6^1,7|run1.2i|424.5|5062|10763.14
g^i^r^u^y^z|20|24:53.56|1493|0|multiband|2.060352|2060352|3,0^3,1^3,2^3,3|run1.2i|715.75|4432|25585.97
g^i^r^u^y^z|20|23:12.74|1392|0|multiband|1.164748|1164748|1,4^1,5^1,6^1,7|run1.2i|666.74|4849|23629.9
g^i^r^u^y^z|20|1:55.35|115|0|multiband|0.521652|521652|6,4^6,5^6,6^6,7|run1.2i|102.32|4848|1339.95
g^i^r^u^y^z|20|6:16.58|376|0|multiband|0.521484|521484|1,4^1,5^1,6^1,7|run1.2i|120.0|4852|1694.56
g^i^r^u^y^z|20|1:24:09|5049|0|multiband|2.189496|2189496|0,0^0,1^0,2^0,3|run1.2i|1915.73|4848|92917.19
g^i^r^u^y^z|20|25:14.88|1514|0|multiband|1.158124|1158124|6,0^6,1^6,2^6,3|run1.2i|769.88|4638|24302.98
g^i^r^u^y^z|20|40:39.95|2439|0|multiband|1.3843239999999999|1384324|6,4^6,5^6,6^6,7|run1.2i|973.86|5064|28771.8
g^i^r^u^y^z|20|6:12.97|372|0|multiband|0.5213679999999999|521368|4,4^4,5^4,6^4,7|run1.2i|122.55|5062|1754.76
g^i^r^u^y^z|20|33:03.64|1983|0|multiband|1.491312|1491312|7,4^7,5^7,6^7,7|run1.2i|652.02|5063|17578.95
g^i^r^u^y^z|20|1:07:22|4042|0|multiband|1.640572|1640572|1,0^1,1^1,2^1,3|run1.2i|1672.32|5065|58656.08
g^i^r^u^y^z|20|1:03:43|3823|0|multiband|1.652384|1652384|4,0^4,1^4,2^4,3|run1.2i|1626.2|5065|54737.64
g^i^r^u^y^z|20|6:12.32|372|0|multiband|0.521532|521532|1,0^1,1^1,2^1,3|run1.2i|120.44|5066|1738.15
g^i^r^u^y^z|20|58:19.37|3499|0|multiband|1.9092600000000002|1909260|1,4^1,5^1,6^1,7|run1.2i|1504.57|4848|65116.86
g^i^r^u^y^z|20|29:49.27|1789|0|multiband|1.05306|1053060|1,4^1,5^1,6^1,7|run1.2i|512.04|4851|13469.06
g^i^r^u^y^z|20|22:49.34|1369|0|multiband|1.157892|1157892|1,4^1,5^1,6^1,7|run1.2i|675.64|4637|21184.98
g^i^r^u^y^z|20|4:14.85|254|0|multiband|0.521532|521532|2,4^2,5^2,6^2,7|run1.2i|114.86|5066|1607.24
g^i^r^u^y^z|20|56:27.63|3387|0|multiband|1.407444|1407444|4,4^4,5^4,6^4,7|run1.2i|1872.93|4431|59268.53
g^i^r^u^y^z|20|27:13.04|1633|0|multiband|1.170496|1170496|1,0^1,1^1,2^1,3|run1.2i|955.52|4638|26100.07
g^i^r^u^y^z|20|56:47.12|3407|0|multiband|1.90312|1903120|0,4^0,5^0,6^0,7|run1.2i|1325.02|4848|63179.16
g^i^r^u^y^z|20|2:06.31|126|0|multiband|0.521532|521532|0,4^0,5^0,6^0,7|run1.2i|96.76|4640|1214.42
g^i^r^u^y^z|20|1:19:18|4758|0|multiband|2.090268|2090268|2,4^2,5^2,6^2,7|run1.2i|2330.5|4432|84497.15
g^i^r^u^y^z|20|32:08.02|1928|0|multiband|1.992944|1992944|2,0^2,1^2,2^2,3|run1.2i|1082.52|4432|33453.03
g^i^r^u^y^z|20|28:04.22|1684|0|multiband|1.1751|1175100|2,0^2,1^2,2^2,3|run1.2i|920.98|4638|27144.11
g^i^r^u^y^z|20|8:02.95|482|0|multiband|1.4108559999999999|1410856|5,0^5,1^5,2^5,3|run1.2i|279.61|4429|7836.32
g^i^r^u^y^z|20|30:22.45|1822|0|multiband|1.17288|1172880|2,4^2,5^2,6^2,7|run1.2i|927.14|4638|29713.36
g^i^r^u^y^z|20|25:23.25|1523|0|multiband|1.6280519999999998|1628052|5,0^5,1^5,2^5,3|run1.2i|750.1|4430|27361.88
g^i^r^u^y^z|20|1:41.62|101|0|multiband|0.5215839999999999|521584|1,4^1,5^1,6^1,7|run1.2i|94.7|4433|1155.52
g^i^r^u^y^z|20|47:01.86|2821|0|multiband|1.41096|1410960|3,4^3,5^3,6^3,7|run1.2i|1487.13|4431|50806.71
g^i^r^u^y^z|20|26:27.59|1587|0|multiband|1.3184|1318400|3,4^3,5^3,6^3,7|run1.2i|901.11|4849|28234.64
g^i^r^u^y^z|20|6:30.01|390|0|multiband|1.371812|1371812|5,0^5,1^5,2^5,3|run1.2i|201.85|4848|5539.32
g^i^r^u^y^z|20|1:07:52|4072|0|multiband|1.59268|1592680|7,0^7,1^7,2^7,3|run1.2i|1838.5|5065|59642.01
g^i^r^u^y^z|20|1:32:33|5553|0|multiband|1.683152|1683152|6,0^6,1^6,2^6,3|run1.2i|2719.79|5064|87839.62
g^i^r^u^y^z|20|43:37.83|2617|0|multiband|2.094332|2094332|4,4^4,5^4,6^4,7|run1.2i|1133.67|4429|47475.41
g^i^r^u^y^z|20|1:12:14|4334|0|multiband|1.677592|1677592|4,0^4,1^4,2^4,3|run1.2i|1927.19|5064|64507.73
g^i^r^u^y^z|20|1:21:11|4871|0|multiband|2.142112|2142112|3,0^3,1^3,2^3,3|run1.2i|1550.1|5063|75126.32
g^i^r^u^y^z|20|1:42:16|6136|0|multiband|2.196564|2196564|1,0^1,1^1,2^1,3|run1.2i|2199.02|5063|98852.07
g^i^r^u^y^z|20|6:13.52|373|0|multiband|0.521524|521524|1,4^1,5^1,6^1,7|run1.2i|118.77|5066|1725.25
g^i^r^u^y^z|20|6:13.79|373|0|multiband|0.5215879999999999|521588|7,0^7,1^7,2^7,3|run1.2i|122.9|5062|1776.89
g^i^r^u^y^z|20|1:31.25|91|0|multiband|0.52166|521660|6,0^6,1^6,2^6,3|run1.2i|84.74|4636|1003.79
g^i^r^u^y^z|20|1:15:06|4506|0|multiband|1.712736|1712736|5,0^5,1^5,2^5,3|run1.2i|2302.22|4640|81407.47
g^i^r^u^y^z|20|6:12.42|372|0|multiband|0.521616|521616|0,4^0,5^0,6^0,7|run1.2i|120.98|5066|1773.03
g^i^r^u^y^z|20|30:17.22|1817|0|multiband|1.114924|1114924|1,0^1,1^1,2^1,3|run1.2i|533.14|4851|13963.15
g^i^r^u^y^z|20|43:48.94|2628|0|multiband|1.398552|1398552|4,0^4,1^4,2^4,3|run1.2i|1407.74|4637|47674.21
g^i^r^u^y^z|20|6:10.40|370|0|multiband|0.5214|521400|0,0^0,1^0,2^0,3|run1.2i|121.28|4852|1753.04
g^i^r^u^y^z|20|28:43.76|1723|0|multiband|1.17406|1174060|4,4^4,5^4,6^4,7|run1.2i|843.53|4638|28120.52
g^i^r^u^y^z|20|40:51.70|2451|0|multiband|1.820764|1820764|4,4^4,5^4,6^4,7|run1.2i|1001.51|5063|28944.6
g^i^r^u^y^z|20|13:06.60|786|0|multiband|1.493828|1493828|1,0^1,1^1,2^1,3|run1.2i|433.54|4432|12232.58
g^i^r^u^y^z|20|35:43.65|2143|0|multiband|1.385464|1385464|0,4^0,5^0,6^0,7|run1.2i|805.62|4431|39089.5
g^i^r^u^y^z|20|1:52.32|112|0|multiband|0.521536|521536|2,0^2,1^2,2^2,3|run1.2i|98.44|4433|1204.66
g^i^r^u^y^z|20|50:01.87|3001|0|multiband|1.409448|1409448|3,0^3,1^3,2^3,3|run1.2i|1692.42|4637|52438.46
g^i^r^u^y^z|20|1:54.28|114|0|multiband|0.5217400000000001|521740|4,0^4,1^4,2^4,3|run1.2i|97.66|4636|1210.48
g^i^r^u^y^z|20|28:17.93|1697|0|multiband|1.175596|1175596|4,4^4,5^4,6^4,7|run1.2i|958.43|4849|29062.85
g^i^r^u^y^z|20|1:09:37|4177|0|multiband|1.570152|1570152|7,0^7,1^7,2^7,3|run1.2i|1854.28|5064|61390.89
g^i^r^u^y^z|20|1:52.40|112|0|multiband|0.5216959999999999|521696|6,4^6,5^6,6^6,7|run1.2i|100.92|4636|1277.87
g^i^r^u^y^z|20|44:37.63|2677|0|multiband|1.631996|1631996|7,4^7,5^7,6^7,7|run1.2i|1259.2|4430|48562.4
g^i^r^u^y^z|20|1:41.37|101|0|multiband|0.52168|521680|5,0^5,1^5,2^5,3|run1.2i|93.88|4433|1130.73
g^i^r^u^y^z|20|1:06:33|3993|0|multiband|1.71222|1712220|6,0^6,1^6,2^6,3|run1.2i|1682.26|4640|72670.34
g^i^r^u^y^z|20|25:24.09|1524|0|multiband|1.334192|1334192|7,0^7,1^7,2^7,3|run1.2i|775.88|4850|25685.7
g^i^r^u^y^z|20|37:05.47|2225|0|multiband|1.409868|1409868|6,0^6,1^6,2^6,3|run1.2i|1258.82|4431|39529.67
g^i^r^u^y^z|20|31:56.20|1916|0|multiband|1.060568|1060568|2,4^2,5^2,6^2,7|run1.2i|536.18|4851|16431.53
g^i^r^u^y^z|20|1:08:53|4133|0|multiband|2.099456|2099456|5,0^5,1^5,2^5,3|run1.2i|1410.61|5063|60764.5
g^i^r^u^y^z|20|1:00:04|3604|0|multiband|2.144368|2144368|3,0^3,1^3,2^3,3|run1.2i|1426.5|4848|65162.83
g^i^r^u^y^z|20|6:04.81|364|0|multiband|0.521552|521552|2,4^2,5^2,6^2,7|run1.2i|121.59|4852|1741.82
g^i^r^u^y^z|20|26:17.63|1577|0|multiband|1.156872|1156872|7,4^7,5^7,6^7,7|run1.2i|811.1|4639|26511.36
g^i^r^u^y^z|20|21:44.74|1304|0|multiband|1.171624|1171624|5,4^5,5^5,6^5,7|run1.2i|711.46|4849|22251.56
g^i^r^u^y^z|20|21:45.51|1305|0|multiband|1.162272|1162272|2,0^2,1^2,2^2,3|run1.2i|737.18|4850|22049.38
g^i^r^u^y^z|20|41:39.81|2499|0|multiband|1.815996|1815996|5,4^5,5^5,6^5,7|run1.2i|1103.66|5066|30101.92
g^i^r^u^y^z|20|45:46.15|2746|0|multiband|1.4187159999999999|1418716|2,0^2,1^2,2^2,3|run1.2i|1454.77|4637|47990.99
g^i^r^u^y^z|20|20:24.85|1224|0|multiband|1.381648|1381648|4,0^4,1^4,2^4,3|run1.2i|718.08|4431|21373.48
g^i^r^u^y^z|20|30:58.86|1858|0|multiband|1.637308|1637308|4,0^4,1^4,2^4,3|run1.2i|1011.89|4430|33537.73
g^i^r^u^y^z|20|1:16:12|4572|0|multiband|1.669592|1669592|1,0^1,1^1,2^1,3|run1.2i|2205.12|5064|68740.94
g^i^r^u^y^z|20|1:39:15|5955|0|multiband|2.209156|2209156|2,0^2,1^2,2^2,3|run1.2i|2129.06|5063|95387.87
g^i^r^u^y^z|20|1:34:40|5680|0|multiband|1.682824|1682824|5,0^5,1^5,2^5,3|run1.2i|2667.33|5064|89744.04
g^i^r^u^y^z|20|47:27.84|2847|0|multiband|2.038896|2038896|6,4^6,5^6,6^6,7|run1.2i|1084.64|5066|36709.78
g^i^r^u^y^z|20|1:05:54|3954|0|multiband|2.11488|2114880|3,4^3,5^3,6^3,7|run1.2i|1644.75|4429|72284.75
g^i^r^u^y^z|20|29:40.01|1780|0|multiband|1.6085319999999999|1608532|0,0^0,1^0,2^0,3|run1.2i|1053.53|4431|30934.93
g^i^r^u^y^z|20|7:39.46|459|0|multiband|1.393568|1393568|1,4^1,5^1,6^1,7|run1.2i|265.95|4636|7171.24
g^i^r^u^y^z|20|22:32.93|1352|0|multiband|1.151864|1151864|3,0^3,1^3,2^3,3|run1.2i|722.24|4639|23001.26
g^i^r^u^y^z|20|18:07.75|1087|0|multiband|1.134888|1134888|7,0^7,1^7,2^7,3|run1.2i|624.89|4849|18247.85
g^i^r^u^y^z|20|25:09.10|1509|0|multiband|1.174768|1174768|3,4^3,5^3,6^3,7|run1.2i|336.59|5066|8381.88
g^i^r^u^y^z|20|33:26.17|2006|0|multiband|1.135292|1135292|2,0^2,1^2,2^2,3|run1.2i|678.39|4851|18590.72
g^i^r^u^y^z|20|29:17.99|1757|0|multiband|1.840808|1840808|3,4^3,5^3,6^3,7|run1.2i|644.24|4848|31455.22
g^i^r^u^y^z|20|1:33:04|5584|0|multiband|2.174788|2174788|0,4^0,5^0,6^0,7|run1.2i|2213.16|4429|102789.45
g^i^r^u^y^z|20|6:03.75|363|0|multiband|0.521592|521592|5,0^5,1^5,2^5,3|run1.2i|120.83|5062|1693.24
g^i^r^u^y^z|20|1:34.45|94|0|multiband|0.521656|521656|3,0^3,1^3,2^3,3|run1.2i|88.07|4636|1122.86
g^i^r^u^y^z|20|1:37.06|97|0|multiband|0.521548|521548|7,4^7,5^7,6^7,7|run1.2i|88.38|4429|1037.18
g^i^r^u^y^z|20|1:00:11|3611|0|multiband|1.708688|1708688|5,4^5,5^5,6^5,7|run1.2i|1701.62|4640|65560.62
g^i^r^u^y^z|20|19:41.58|1181|0|multiband|1.146964|1146964|4,0^4,1^4,2^4,3|run1.2i|541.77|4639|20184.62
g^i^r^u^y^z|20|6:10.10|370|0|multiband|0.52146|521460|3,0^3,1^3,2^3,3|run1.2i|121.62|4852|1726.63
g^i^r^u^y^z|20|21:20.02|1280|0|multiband|1.162056|1162056|4,4^4,5^4,6^4,7|run1.2i|679.58|4639|21192.24
g^i^r^u^y^z|20|18:40.36|1120|0|multiband|1.152496|1152496|2,4^2,5^2,6^2,7|run1.2i|576.88|4639|18888.88
g^i^r^u^y^z|20|1:09:52|4192|0|multiband|1.719176|1719176|2,4^2,5^2,6^2,7|run1.2i|1965.13|4430|76220.6
g^i^r^u^y^z|20|1:24:04|5044|0|multiband|2.176004|2176004|1,4^1,5^1,6^1,7|run1.2i|2101.05|4429|92669.12
g^i^r^u^y^z|20|1:16:31|4591|0|multiband|2.094036|2094036|7,0^7,1^7,2^7,3|run1.2i|1565.73|5066|69677.08
g^i^r^u^y^z|20|18:56.52|1136|0|multiband|1.147764|1147764|0,0^0,1^0,2^0,3|run1.2i|666.04|4639|20359.38
g^i^r^u^y^z|20|6:13.17|373|0|multiband|0.521568|521568|2,0^2,1^2,2^2,3|run1.2i|123.52|5066|1735.72
g^i^r^u^y^z|20|36:45.21|2205|0|multiband|1.178048|1178048|1,0^1,1^1,2^1,3|run1.2i|1319.08|4849|38336.69
g^i^r^u^y^z|20|35:00.77|2100|0|multiband|1.139672|1139672|7,0^7,1^7,2^7,3|run1.2i|700.49|4851|20809.55
g^i^r^u^y^z|20|19:31.17|1171|0|multiband|1.176852|1176852|5,4^5,5^5,6^5,7|run1.2i|572.4|4850|19121.82
g^i^r^u^y^z|20|28:07.57|1687|0|multiband|1.057988|1057988|1,4^1,5^1,6^1,7|run1.2i|453.16|5065|12357.42
g^i^r^u^y^z|20|6:13.56|373|0|multiband|0.521536|521536|1,0^1,1^1,2^1,3|run1.2i|125.46|4852|1772.77
g^i^r^u^y^z|20|20:45.90|1245|0|multiband|1.151908|1151908|1,0^1,1^1,2^1,3|run1.2i|644.57|4850|21195.68
g^i^r^u^y^z|20|1:52.49|112|0|multiband|0.5216919999999999|521692|2,4^2,5^2,6^2,7|run1.2i|92.6|4636|1180.01
g^i^r^u^y^z|20|6:14.79|374|0|multiband|0.521528|521528|2,4^2,5^2,6^2,7|run1.2i|124.15|5062|1808.88
g^i^r^u^y^z|20|25:27.15|1527|0|multiband|1.164788|1164788|4,0^4,1^4,2^4,3|run1.2i|850.39|4638|26706.01
g^i^r^u^y^z|20|1:07:56|4076|0|multiband|2.198576|2198576|2,0^2,1^2,2^2,3|run1.2i|1354.57|4848|75013.39
g^i^r^u^y^z|20|21:42.70|1302|0|multiband|1.152572|1152572|1,4^1,5^1,6^1,7|run1.2i|617.64|4639|22460.8
g^i^r^u^y^z|20|39:44.95|2384|0|multiband|1.385112|1385112|4,4^4,5^4,6^4,7|run1.2i|1006.96|5064|28397.51
g^i^r^u^y^z|20|15:13.65|913|0|multiband|0.9294800000000001|929480|2,0^2,1^2,2^2,3|run1.2i|222.69|5062|4610.5
g^i^r^u^y^z|20|6:15.81|375|0|multiband|0.521408|521408|5,4^5,5^5,6^5,7|run1.2i|124.18|4852|1825.56
g^i^r^u^y^z|20|37:31.33|2251|0|multiband|1.146488|1146488|6,4^6,5^6,6^6,7|run1.2i|773.88|4851|23084.66
g^i^r^u^y^z|20|31:56.20|1916|0|multiband|1.124936|1124936|5,4^5,5^5,6^5,7|run1.2i|633.05|5065|16491.18
g^i^r^u^y^z|20|26:47.73|1607|0|multiband|1.165008|1165008|6,0^6,1^6,2^6,3|run1.2i|691.66|4639|27089.05
g^i^r^u^y^z|20|1:41.02|101|0|multiband|0.521548|521548|3,0^3,1^3,2^3,3|run1.2i|93.32|4433|1229.58
g^i^r^u^y^z|20|1:45.63|105|0|multiband|0.52164|521640|0,4^0,5^0,6^0,7|run1.2i|93.78|4433|1131.44
g^i^r^u^y^z|20|36:19.10|2179|0|multiband|1.150452|1150452|5,0^5,1^5,2^5,3|run1.2i|701.07|4851|21462.91
g^i^r^u^y^z|20|32:56.40|1976|0|multiband|1.194264|1194264|1,4^1,5^1,6^1,7|run1.2i|1011.76|4638|32467.16
g^i^r^u^y^z|20|37:40.31|2260|0|multiband|1.5889600000000002|1588960|5,4^5,5^5,6^5,7|run1.2i|875.25|5063|23519.68
g^i^r^u^y^z|20|24:53.42|1493|0|multiband|1.163964|1163964|0,0^0,1^0,2^0,3|run1.2i|688.81|4849|25436.99
g^i^r^u^y^z|20|6:42.88|402|0|multiband|1.389208|1389208|2,4^2,5^2,6^2,7|run1.2i|193.34|4640|5257.66
g^i^r^u^y^z|20|28:08.74|1688|0|multiband|1.169624|1169624|2,4^2,5^2,6^2,7|run1.2i|936.15|4850|28979.52
g^i^r^u^y^z|20|36:25.54|2185|0|multiband|2.099272|2099272|1,0^1,1^1,2^1,3|run1.2i|1144.84|4429|39412.08
g^i^r^u^y^z|20|1:44.65|104|0|multiband|0.521484|521484|3,4^3,5^3,6^3,7|run1.2i|92.22|4433|1122.92
g^i^r^u^y^z|20|6:14.03|374|0|multiband|0.521528|521528|2,0^2,1^2,2^2,3|run1.2i|121.78|4852|1796.4
g^i^r^u^y^z|20|2:10.32|130|0|multiband|0.521528|521528|1,0^1,1^1,2^1,3|run1.2i|96.42|4640|1195.64
g^i^r^u^y^z|20|18:53.81|1133|0|multiband|1.130912|1130912|0,4^0,5^0,6^0,7|run1.2i|562.6|4850|18423.82
g^i^r^u^y^z|20|28:46.23|1726|0|multiband|1.64004|1640040|3,0^3,1^3,2^3,3|run1.2i|946.63|4430|30841.44
g^i^r^u^y^z|20|41:41.82|2501|0|multiband|1.398232|1398232|1,4^1,5^1,6^1,7|run1.2i|1316.69|4431|45264.95
g^i^r^u^y^z|20|44:30.60|2670|0|multiband|1.386612|1386612|5,4^5,5^5,6^5,7|run1.2i|1146.75|5064|33502.34
g^i^r^u^y^z|20|32:46.18|1966|0|multiband|1.366272|1366272|7,0^7,1^7,2^7,3|run1.2i|1113.38|4431|33944.79
g^i^r^u^y^z|20|1:07:28|4048|0|multiband|1.6881119999999998|1688112|5,4^5,5^5,6^5,7|run1.2i|1846.45|4430|73978.62
g^i^r^u^y^z|20|1:36.29|96|0|multiband|0.521548|521548|7,0^7,1^7,2^7,3|run1.2i|89.56|4429|1056.34
g^i^r^u^y^z|20|20:03.88|1203|0|multiband|1.169972|1169972|3,4^3,5^3,6^3,7|run1.2i|631.36|4639|20582.55
g^i^r^u^y^z|20|1:46.47|106|0|multiband|0.5216959999999999|521696|0,0^0,1^0,2^0,3|run1.2i|93.94|4433|1125.75
g^i^r^u^y^z|20|33:01.90|1981|0|multiband|1.138032|1138032|3,0^3,1^3,2^3,3|run1.2i|605.82|4851|17431.22
g^i^r^u^y^z|20|21:15.79|1275|0|multiband|1.14872|1148720|3,4^3,5^3,6^3,7|run1.2i|668.32|4637|19971.49
g^i^r^u^y^z|20|6:20.78|380|0|multiband|1.169248|1169248|5,4^5,5^5,6^5,7|run1.2i|170.72|4848|4487.5
g^i^r^u^y^z|20|23:51.53|1431|0|multiband|1.157104|1157104|4,4^4,5^4,6^4,7|run1.2i|772.83|4850|23783.82
g^i^r^u^y^z|20|1:22:18|4938|0|multiband|1.721412|1721412|7,4^7,5^7,6^7,7|run1.2i|2248.18|4640|89984.99
g^i^r^u^y^z|20|22:58.35|1378|0|multiband|1.152044|1152044|7,4^7,5^7,6^7,7|run1.2i|642.15|4638|21605.56
g^i^r^u^y^z|20|40:37.80|2437|0|multiband|1.651756|1651756|2,0^2,1^2,2^2,3|run1.2i|1402.77|4430|44060.46
g^i^r^u^y^z|20|48:39.88|2919|0|multiband|1.690404|1690404|4,4^4,5^4,6^4,7|run1.2i|1296.17|4430|53369.86
